In [2]:
!pip install neo4j

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.7/301.7 kB 10.6 MB/s eta 0:00:00


In [37]:
from neo4j import GraphDatabase

# Connection details
URI = "neo4j://38.242.232.192:7687"
AUTH = ("nest", "neurons-newbie")

# Query to view existing nodes
query = """
MATCH (n)
RETURN n
LIMIT 10
"""

# Connect to the database and run the query
try:
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        with driver.session() as session:
            result = session.run(query)

            # Process and print the nodes
            print("Nodes in the database:")
            for record in result:
                print(record["n"])  # Access the node data
except Exception as e:
    print(f"An error occurred: {e}")

Nodes in the database:
<Node element_id='4:4e1a6d5f-d162-4464-a50b-c8bfba1745ea:3152' labels=frozenset({'SubjectNode'}) properties={'name': 'NCT00752622'}>
<Node element_id='4:4e1a6d5f-d162-4464-a50b-c8bfba1745ea:3153' labels=frozenset({'ObjectNode'}) properties={'name': "'Crohn's Disease'"}>
<Node element_id='4:4e1a6d5f-d162-4464-a50b-c8bfba1745ea:3154' labels=frozenset({'ObjectNode'}) properties={'name': "'Infliximab'"}>
<Node element_id='4:4e1a6d5f-d162-4464-a50b-c8bfba1745ea:3155' labels=frozenset({'ObjectNode'}) properties={'name': "'CDAI score change'"}>
<Node element_id='4:4e1a6d5f-d162-4464-a50b-c8bfba1745ea:3156' labels=frozenset({'ObjectNode'}) properties={'name': "'Harvey-Bradshaw Index'"}>
<Node element_id='4:4e1a6d5f-d162-4464-a50b-c8bfba1745ea:3157' labels=frozenset({'ObjectNode'}) properties={'name': " 'Disease progression/lack of response'"}>
<Node element_id='4:4e1a6d5f-d162-4464-a50b-c8bfba1745ea:3158' labels=frozenset({'SubjectNode'}) properties={'name': 'NCT02177422

In [38]:
# Queries to count nodes and relationships
query_nodes = "MATCH (n) RETURN COUNT(n) AS TotalNodes;"
query_relationships = "MATCH ()-[r]->() RETURN COUNT(r) AS TotalRelationships;"

try:
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        with driver.session() as session:
            # Count nodes
            result_nodes = session.run(query_nodes)
            for record in result_nodes:
                print("Total Nodes in the Database:", record["TotalNodes"])

            # Count relationships
            result_relationships = session.run(query_relationships)
            for record in result_relationships:
                print("Total Relationships in the Database:", record["TotalRelationships"])
except Exception as e:
    print(f"An error occurred: {e}")


Total Nodes in the Database: 59
Total Relationships in the Database: 59


In [8]:
try:
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        with driver.session() as session:
            result = session.run("CALL gds.graph.list()")

            # Process and print the results
            for record in result:
                print(record)
except Exception as e:
    print(f"An error occurred: {e}")

In [39]:
# Query to fetch distinct labels
def fetch_labels(tx):
    query = """
    MATCH (n) RETURN DISTINCT labels(n) AS nodeLabels
    """
    result = tx.run(query)
    return [record["nodeLabels"] for record in result]

# Main function to connect and fetch labels
def main():
    driver = GraphDatabase.driver(URI, auth=AUTH)

    try:
        with driver.session() as session:
            labels = session.read_transaction(fetch_labels)
            print("Distinct Labels in the Database:")
            for label in labels:
                print(label)
    except Exception as e:
        print(f"Error: {e}")
    finally:
        driver.close()

if __name__ == "__main__":
    main()


Distinct Labels in the Database:
['SubjectNode']
['ObjectNode']


<ipython-input-39-8feb4cda7f53>:15: DeprecationWarning: read_transaction has been renamed to execute_read
  labels = session.read_transaction(fetch_labels)


In [40]:
def fetch_relationship_types(tx):
    query = """
    CALL db.relationshipTypes() YIELD relationshipType
    RETURN relationshipType
    """
    result = tx.run(query)
    return [record["relationshipType"] for record in result]

# Main function to connect and fetch relationship types
def main():
    driver = GraphDatabase.driver(URI, auth=AUTH)

    try:
        with driver.session() as session:
            relationships = session.read_transaction(fetch_relationship_types)
            print("Distinct Relationship Types in the Database:")
            for relationship in relationships:
                print(relationship)
    except Exception as e:
        print(f"Error: {e}")
    finally:
        driver.close()

if __name__ == "__main__":
    main()

Distinct Relationship Types in the Database:
INVOLVES
EVALUATES
MEASURES_PRIMARY
MEASURES_SECONDARY
HAS_CRITERIA
${relationship_type}


<ipython-input-40-21312d34543f>:15: DeprecationWarning: read_transaction has been renamed to execute_read
  relationships = session.read_transaction(fetch_relationship_types)


In [42]:
def is_graph_projected(tx):
    query = """
    CALL gds.graph.exists('clinicalTrialsGraph')
    YIELD exists
    RETURN exists
    """
    result = tx.run(query)
    return result.single()["exists"]

# Function to project the graph
def project_graph(tx):
    query = """
    CALL gds.graph.project(
        'clinicalTrialsGraph',
        ['SubjectNode', 'ObjectNode'],  -- Use the distinct labels
        {
            INVOLVES: {},
            EVALUATES: {},
            MEASURES_PRIMARY: {},
            MEASURES_SECONDARY: {},
            HAS_CRITERIA: {}
        }  -- Use the distinct relationship types
    )
    """
    tx.run(query)

# Function to run node similarity
def find_similar_trials(tx, trial_id):
    query = f"""
    CALL gds.nodeSimilarity.stream('clinicalTrialsGraph')
    YIELD node1, node2, similarity
    WHERE gds.util.asNode(node1).id = '{trial_id}'
    RETURN gds.util.asNode(node2).id AS similarTrial, similarity
    ORDER BY similarity DESC
    LIMIT 10
    """
    result = tx.run(query)
    return [{"trial": record["similarTrial"], "similarity": record["similarity"]} for record in result]

# Main script
def main():
    driver = GraphDatabase.driver(URI, auth=AUTH)

    with driver.session() as session:
        # Check if the graph is already projected
        if not session.execute_read(is_graph_projected):  # Use `execute_read` for compatibility
            print("Graph not projected. Projecting now...")
            session.execute_write(project_graph)  # Use `execute_write` for compatibility
        else:
            print("Graph is already projected.")

        # Get input from user
        trial_id = input("Enter the trial ID (e.g., NCT00752622): ")

        # Find similar trials
        print(f"Finding trials similar to {trial_id}...")
        similar_trials = session.execute_read(find_similar_trials, trial_id)

        if similar_trials:
            print("Top 10 similar trials:")
            for i, trial in enumerate(similar_trials, 1):
                print(f"{i}. Trial ID: {trial['trial']}, Similarity: {trial['similarity']:.4f}")
        else:
            print("No similar trials found.")

    driver.close()

if __name__ == "__main__":
    main()

Graph is already projected.
Enter the trial ID (e.g., NCT00752622): NCT01399125
Finding trials similar to NCT01399125...
No similar trials found.


In [27]:
def main():
    try:
        driver = GraphDatabase.driver(URI, auth=AUTH)
        with driver.session() as session:
            print("Connected to Neo4j successfully!")
            # Run your queries here...
    except ServiceUnavailable as e:
        print(f"Failed to connect to Neo4j: {e}")
    finally:
        driver.close()

if __name__ == "__main__":
    main()


Connected to Neo4j successfully!


In [51]:
# Function to fetch all nodes in the graph
def fetch_nodes(tx):
    query = """
    MATCH (n)
    RETURN labels(n) AS labels, n.name AS name
    """
    result = tx.run(query)
    return [{"labels": record["labels"], "name": record["name"]} for record in result]

# Main function to connect to Neo4j and retrieve nodes
def main():
    driver = GraphDatabase.driver(URI, auth=AUTH)

    try:
        with driver.session() as session:
            nodes = session.read_transaction(fetch_nodes)
            if nodes:
                print("Nodes present in the graph:")
                for node in nodes:
                    print(f"Labels: {node['labels']}, Name: {node['name']}")
            else:
                print("No nodes found in the graph.")
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        driver.close()

if __name__ == "__main__":
    main()

Nodes present in the graph:
Labels: ['SubjectNode'], Name: NCT00752622
Labels: ['ObjectNode'], Name: 'Crohn's Disease'
Labels: ['ObjectNode'], Name: 'Infliximab'
Labels: ['ObjectNode'], Name: 'CDAI score change'
Labels: ['ObjectNode'], Name: 'Harvey-Bradshaw Index'
Labels: ['ObjectNode'], Name:  'Disease progression/lack of response'
Labels: ['SubjectNode'], Name: NCT02177422
Labels: ['ObjectNode'], Name: 'Telmisartan'
Labels: ['ObjectNode'], Name: 'Goal blood pressure'
Labels: ['ObjectNode'], Name: Heart rate'
Labels: ['ObjectNode'], Name: Mean diastolic ≥ 95 mmHg'
Labels: ['SubjectNode'], Name: NCT06082986
Labels: ['ObjectNode'], Name: 'Endoscopic Remission'
Labels: ['ObjectNode'], Name: 'Clinical Remission'
Labels: ['ObjectNode'], Name:  'Bio-naive Participants'
Labels: ['SubjectNode'], Name: NCT01277822
Labels: ['ObjectNode'], Name: 'Losartan + Amlodipine'
Labels: ['ObjectNode'], Name: 'MSDBP change at Week 8'
Labels: ['ObjectNode'], Name: 'Percentage of participants achieving targ

<ipython-input-51-4a940ca6ab8e>:16: DeprecationWarning: read_transaction has been renamed to execute_read
  nodes = session.read_transaction(fetch_nodes)


In [62]:
def is_graph_projected(tx):
    query = """
    CALL gds.graph.exists('clinicalTrialsGraph')
    YIELD exists
    RETURN exists
    """
    result = tx.run(query)
    return result.single()["exists"]

# Function to project the graph
def project_graph(tx):
    query = """
    CALL gds.graph.project(
        'clinicalTrialsGraph',
        ['SubjectNode', 'ObjectNode'],
        {
            INVOLVES: {orientation: 'UNDIRECTED'},
            EVALUATES: {orientation: 'UNDIRECTED'},
            MEASURES_PRIMARY: {orientation: 'UNDIRECTED'},
            MEASURES_SECONDARY: {orientation: 'UNDIRECTED'},
            HAS_CRITERIA: {orientation: 'UNDIRECTED'}
        }
    )
    """
    tx.run(query)
    print("Graph successfully projected with undirected relationships.")

# Function to drop and re-project the graph
def reproject_graph(tx):
    query = """
    CALL gds.graph.drop('clinicalTrialsGraph', false)
    """
    tx.run(query)
    print("Dropped existing graph projection.")
    project_graph(tx)
    print("Graph re-projected successfully.")

# Function to find similar trials using GDS node similarity
def find_similar_trials(tx, trial_id):
    query = f"""
    CALL gds.nodeSimilarity.stream('clinicalTrialsGraph')
    YIELD node1, node2, similarity
    WHERE gds.util.asNode(node1).name = $trial_id
      AND gds.util.asNode(node1):SubjectNode
      AND gds.util.asNode(node2):SubjectNode
    RETURN gds.util.asNode(node2).name AS similarTrial, similarity
    ORDER BY similarity DESC
    LIMIT 10
    """
    print(f"Executing similarity query for trial ID: {trial_id}")
    result = tx.run(query, trial_id=trial_id)
    results = [{"trial": record["similarTrial"], "similarity": record["similarity"]} for record in result]
    print(f"Results: {results}")
    return results

# Function to check if a node exists in the graph
def check_node_exists(tx, node_name, label):
    query = f"""
    MATCH (n:{label} {{name: TRIM($node_name)}})
    RETURN COUNT(n) > 0 AS exists
    """
    print(f"Checking existence of node: {node_name.strip()}")
    result = tx.run(query, node_name=node_name.strip())
    exists = result.single()["exists"]
    print(f"Node exists: {exists}")
    return exists

# Main script to query for similar trials
def main():
    driver = GraphDatabase.driver(URI, auth=AUTH)

    with driver.session() as session:
        # Re-project the graph to ensure the correct structure
        reproject_graph(session)

        # Get input from user
        trial_id = input("Enter the trial ID (e.g., NCT00752622): ")

        # Check if the node exists
        node_exists = session.read_transaction(check_node_exists, trial_id, 'SubjectNode')
        if not node_exists:
            print(f"Node '{trial_id}' does not exist in the graph. Please enter a valid trial ID.")
            return

        # Find similar trials
        print(f"Finding trials similar to {trial_id}...")
        similar_trials = session.read_transaction(find_similar_trials, trial_id)

        if similar_trials:
            print("Top 10 similar trials:")
            for i, trial in enumerate(similar_trials, 1):
                print(f"{i}. Trial ID: {trial['trial']}, Similarity: {trial['similarity']:.4f}")
        else:
            print("No similar trials found.")
    driver.close()

if __name__ == "__main__":
    main()

Dropped existing graph projection.
Graph successfully projected with undirected relationships.
Graph re-projected successfully.
Enter the trial ID (e.g., NCT00752622): NCT01399125
Checking existence of node: NCT01399125
Node exists: True
Finding trials similar to NCT01399125...
Executing similarity query for trial ID: NCT01399125
Results: [{'trial': 'NCT03114657', 'similarity': 0.25}, {'trial': 'NCT02972658', 'similarity': 0.14285714285714285}, {'trial': 'NCT02791958', 'similarity': 0.1111111111111111}]
Top 10 similar trials:
1. Trial ID: NCT03114657, Similarity: 0.2500
2. Trial ID: NCT02972658, Similarity: 0.1429
3. Trial ID: NCT02791958, Similarity: 0.1111


<ipython-input-62-39c2aca5816c>:80: DeprecationWarning: read_transaction has been renamed to execute_read
  node_exists = session.read_transaction(check_node_exists, trial_id, 'SubjectNode')
<ipython-input-62-39c2aca5816c>:87: DeprecationWarning: read_transaction has been renamed to execute_read
  similar_trials = session.read_transaction(find_similar_trials, trial_id)
